# Obtaining High Performance Areas (C4 model)

## Imports

In [12]:
from cobra.io import read_sbml_model
from cobra import flux_analysis
import random
import time
from cobra.flux_analysis import flux_variability_analysis
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Reading and preparing the model

In [13]:
"""
Read and prepare C4 model
"""

c4_model = read_sbml_model("c4_model.xml")

c4_model.solver = "glpk"

c4_model.objective = '[B]_Maize_biomass_tx'

def c4_maintenance(c4_model):
    #Constrains for light dependent maintenance costs
    atp_b = c4_model.reactions.get_by_id("[B]_ATPase_tx")
    photon_b = c4_model.reactions.get_by_id("[B]_Photon_tx")
    atp_m = c4_model.reactions.get_by_id("[M]_ATPase_tx")
    photon_m = c4_model.reactions.get_by_id("[M]_Photon_tx")

    const_b = c4_model.problem.Constraint((0.0049 * photon_b.flux_expression + 2.7852) - atp_b.flux_expression , lb = 0, ub = 0)
    c4_model.add_cons_vars(const_b)

    const_m = c4_model.problem.Constraint((0.0049 * photon_m.flux_expression + 2.7852) - atp_m.flux_expression , lb = 0, ub = 0)
    c4_model.add_cons_vars(const_m)
    # ATP/NADPH 3:1 constraints
    const = c4_model.problem.Constraint(c4_model.reactions.get_by_id("[B]_ATPase_tx").flux_expression -  3 * (c4_model.reactions.get_by_id("[B]_NADPHoxc_tx").flux_expression + c4_model.reactions.get_by_id("[B]_NADPHoxp_tx").flux_expression + c4_model.reactions.get_by_id("[B]_NADPHoxm_tx").flux_expression) , lb = 0, ub = 0)
    c4_model.add_cons_vars(const)

    const = c4_model.problem.Constraint(c4_model.reactions.get_by_id("[M]_ATPase_tx").flux_expression -  3 * (c4_model.reactions.get_by_id("[M]_NADPHoxc_tx").flux_expression + c4_model.reactions.get_by_id("[M]_NADPHoxp_tx").flux_expression + c4_model.reactions.get_by_id("[M]_NADPHoxm_tx").flux_expression) , lb = 0, ub = 0)
    c4_model.add_cons_vars(const)

#Add Light dependent maintenance
c4_maintenance(c4_model)


#Ensuring directionality of uptake reaction
c4_model.reactions.get_by_id("[B]_Nitrate_tx").bounds = (0, 999999)
c4_model.reactions.get_by_id("[M]_Nitrate_tx").bounds = (0, 999999)
c4_model.reactions.get_by_id("[B]_Photon_tx").bounds = (0, 999999)
c4_model.reactions.get_by_id("[M]_Photon_tx").bounds = (0, 999999)


No objective coefficients in model. Unclear what should be optimized


## Performing pFBA

### Determination of biomass flux and absolute sum of fluxes for each of the 9 starting points

In [14]:
"""
Functions to perform the simulation
"""

def c4_simulation(light, N, c4_model):
    ##C4
    with c4_model:
        #Light Uptale constrain
        B_Im_hnu = c4_model.reactions.get_by_id("[B]_Photon_tx")
        M_Im_hnu = c4_model.reactions.get_by_id("[M]_Photon_tx")
        #CONSTRAINT: Total Photon uptake limited to "light" µE
        const_hnu_sum = c4_model.problem.Constraint( B_Im_hnu.flux_expression + M_Im_hnu.flux_expression,
                                                lb = 0, ub = light)
        c4_model.add_cons_vars(const_hnu_sum)
        #CONSTRAINT: Total Photon uptake by bundle sheath must be less or equal than in mesophyll
        const_hnu_ratio = c4_model.problem.Constraint( M_Im_hnu.flux_expression - B_Im_hnu.flux_expression,
                                                lb = 0, ub = light)
        c4_model.add_cons_vars(const_hnu_ratio)
        #CONSTRAINT : Total N uptake must not surpass defined upper bound
        m_n = c4_model.reactions.get_by_id("[B]_Nitrate_tx")
        bs_n = c4_model.reactions.get_by_id("[M]_Nitrate_tx")
        const_n_ratio = c4_model.problem.Constraint( bs_n.flux_expression + m_n.flux_expression,
                                               lb = 0, ub = N)
        c4_model.add_cons_vars(const_n_ratio)
        #pFBA
        solution = flux_analysis.pfba(c4_model)
        sum_of_flux = solution.fluxes.abs().sum()
        solution_frame=solution.to_frame()
        growth = solution_frame.loc["[B]_Maize_biomass_tx"]["fluxes"]
        return (sum_of_flux, growth)

In [15]:
"""
Build Dictionary with the combination of Light/Nitrogen conditions as a key,
 and Biomass flux and Sum of fluxes as values
"""

#Grid of light and Nitrogen conditions to use
light_uptake = [100, 250, 500]
nitrogen_uptake = [0.2, 2.5, 6]

#Dictionaries to hold the results
res_c4 = {}

#Perform the simulations for diferent combinations of Light/N and store them in the dictionary
for l in light_uptake:
    for n in nitrogen_uptake:
        res_c4[(l,n)] = c4_simulation(l,n,c4_model)

print(res_c4)



{(100, 0.2): (307.44209419816093, 0.003956163833772806), (100, 2.5): (693.4916382201047, 0.010251688843162948), (100, 6): (693.4916382201047, 0.010251688843162948), (250, 0.2): (307.4420941981622, 0.00395616383377281), (250, 2.5): (1749.0675978769336, 0.027368307237520158), (250, 6): (1749.0675978769336, 0.027368307237520158), (500, 0.2): (307.4420941981622, 0.00395616383377281), (500, 2.5): (3012.6684876816544, 0.049452047922160326), (500, 6): (3509.804113505037, 0.05589600456144867)}


## Computation of the High Performance Areas

In order to compute the high performance area for a starting point a technique involving random sampling was devised in order to map out this area. Firstly, flux balance analysis (FVA) was used to estimate the range of possible nitrogen and light uptake values, with the FVA constrained to have at least a sum of fluxes equal to the absolute sum of fluxes of the starting point. This will save us a lot of computation time, by defining a priori the bounds in which random values for nitrogen and light uptake should be generated.

Then, a new pFBA simulation will be performed with the constrains that the biomass value must be at least 80% of the one obtained with the initial pFBA simulation for the starting point, and the sum of fluxes value cannot surpass the one determined for the starting point.

By defining the high performance areas as the range of feasible solutions for the model under the aforementioned constrains for different light and nitrogen uptakes, one of the most logical courses of action in order to map them out would be to perform a lot of simulations with different light and nitrogen uptakes, and store feasible solutions. That is exactly what the code described below performs.

The function will receive the feasible range of uptakes of nitrogen and light from the FVA and use them in functions from the random module of python, which will generate random numbers within those constrains. These values will then be used as a forced flux value for the nitrogen and light uptakes of the simulation. Unfeasible solutions will be ignored, and the algorithm will stop once 500 feasible solutions have been reached.

The are differences in the implementation of this methodology in the C4 model when compared with the C3 model. The main issue is that there are two possible uptake reactions for nitrogen and light each: one in the bundle sheath and one in the mesophyll. 

This adds another level of complexity when using FVA to compute the uptake bounds, as both reactions must be taken into acccount in order to determine total uptake. As such, another function was created in order to handle FVA results and attempt to establish the high performance area bounds for the random simulations. This was only partially successfull though. 

The FVA determined bounds were able to pinpoint the approximate location of the respective high performance area in the 2D solution space, but it wasn't able to fully encompass it. In order to determine the true high performance are, the FVA results were used as a starting point, and then manually expanded untill no more points were included into the HPA. 

In [16]:
"""
Function to perform the FVA at 80% fraction of optimum, restraining the 
sum of fluxes to be at maximum equal to the sum of fluxes of the initial point
"""

def fva_c4(sum_of_flux, c4_model):
        if sum_of_flux is not None:
        #Sum of flux constrain
            coefficients = dict()
            for rxn in c4_model.reactions:
                coefficients[rxn.forward_variable] = 1.
                coefficients[rxn.reverse_variable] = 1.
            constraint = c4_model.problem.Constraint(0, lb=0, ub=sum_of_flux)

            c4_model.add_cons_vars(constraint)
            c4_model.solver.update()
            constraint.set_linear_coefficients(coefficients=coefficients)

        #FVA
        solution = flux_variability_analysis(c4_model, reaction_list=["[B]_Nitrate_tx","[M]_Nitrate_tx", "[B]_Photon_tx", "[M]_Photon_tx"], fraction_of_optimum=0.8)
        return solution


In [17]:
"""
Function to handle FVA outputs
"""

def starting_conditions(initial_point):
    
    n_list = ["[B]_Nitrate_tx","[M]_Nitrate_tx"]
    l_list = ["[B]_Photon_tx", "[M]_Photon_tx"]

    min_n = 99999
    max_n = 0
    min_l = 99999
    max_l = 0
    
    #Handling nitrogen bounds
    for r in n_list:
        with c4_model:
            c4_model.reactions.get_by_id(r).bounds = (0,0)
            res = fva_c4(sum_of_flux =  initial_point[0], c4_model=c4_model)
            if res.iloc[0]["minimum"] < min_n or res.iloc[1]["minimum"] < min_n:
                minimum = max(res.iloc[0]["minimum"], res.iloc[1]["minimum"])
                min_n = min(minimum, min_n)
            if res.iloc[0]["maximum"] > max_n or res.iloc[1]["maximum"] > max_n:
                maximum = max(res.iloc[0]["maximum"], res.iloc[1]["maximum"] )
                max_n = maximum
    
    #Handling light bounds
    for r in l_list:
        with c4_model:
            c4_model.reactions.get_by_id(r).bounds = (0,0)
            res = fva_c4(sum_of_flux =  initial_point[0], c4_model=c4_model)
            if res.iloc[2]["minimum"] < min_l or res.iloc[3]["minimum"] < min_l:
                minimum = max(res.iloc[2]["minimum"], res.iloc[3]["minimum"])
                min_l = min(minimum, min_l)
            if res.iloc[2]["maximum"] > max_l or res.iloc[3]["maximum"] > max_l:
                maximum = max(res.iloc[2]["maximum"], res.iloc[3]["maximum"])
                max_l = maximum
    
    return min_n, max_n, min_l, max_l


In [18]:
"""
Function to perform the sampling using random points in a uniform distribution
"""


def random_simul_c4(light_lb, light_ub, nitrogen_lb, nitrogen_ub, sum_of_flux, biomass, c4_model):
    with c4_model:
        if sum_of_flux is not None:
        #Sum of flux constrain
            coefficients = dict()
            for rxn in c4_model.reactions:
                coefficients[rxn.forward_variable] = 1.
                coefficients[rxn.reverse_variable] = 1.
            constraint = c4_model.problem.Constraint(0, lb=0, ub=sum_of_flux)

            c4_model.add_cons_vars(constraint)
            c4_model.solver.update()
            constraint.set_linear_coefficients(coefficients=coefficients)

        #Bounds for Biomass
        c4_model.reactions.get_by_id("[B]_Maize_biomass_tx").lower_bound = biomass * 0.8
        c4_model.reactions.get_by_id("[B]_Maize_biomass_tx").upper_bound = 9999

        #Bounds for light
        random_light = random.uniform(light_lb, light_ub)

        #Light Uptake constrain
        B_Im_hnu = c4_model.reactions.get_by_id("[B]_Photon_tx")
        B_Im_hnu.bounds = (0, 9999999)


        M_Im_hnu = c4_model.reactions.get_by_id("[M]_Photon_tx")
        M_Im_hnu.bounds = (0, 99999999)

        #CONSTRAINT: Total Photon uptake limited to "light" µE
        const_hnu_sum = c4_model.problem.Constraint( B_Im_hnu.flux_expression + M_Im_hnu.flux_expression,
                                                lb = random_light, ub = random_light)
        c4_model.add_cons_vars(const_hnu_sum)
        #CONSTRAINT: Total Photon uptake by bundle sheath must be less or equal than in mesophyll
        const_hnu_ratio = c4_model.problem.Constraint( M_Im_hnu.flux_expression - B_Im_hnu.flux_expression,
                                                lb = 0, ub = random_light)
        c4_model.add_cons_vars(const_hnu_ratio)

        #Bounds for N
        random_nitrogen = random.uniform(nitrogen_lb, nitrogen_ub)

        #CONSTRAINT : Total N uptake must not surpass defined upper bound
        bs_n = c4_model.reactions.get_by_id("[B]_Nitrate_tx")
        bs_n.bounds = (0, 9999999)

        m_n = c4_model.reactions.get_by_id("[M]_Nitrate_tx")
        m_n.bounds = (0, 9999999)

        const_n_ratio = c4_model.problem.Constraint( bs_n.flux_expression + m_n.flux_expression,
                                               lb = random_nitrogen, ub = random_nitrogen )
        c4_model.add_cons_vars(const_n_ratio)

        #pFBA
        try:
            solution = flux_analysis.pfba(c4_model)
            solution_frame=solution.to_frame()
            growth = solution_frame.loc["[B]_Maize_biomass_tx"]["fluxes"]
            nitrogen_uptake = solution_frame.loc["[B]_Nitrate_tx"]["fluxes"] + solution_frame.loc["[M]_Nitrate_tx"]["fluxes"]
            light_uptake = solution_frame.loc["[M]_Photon_tx"]["fluxes"] + solution_frame.loc["[B]_Photon_tx"]["fluxes"]
            sum_of_fluxes= solution.fluxes.abs().sum()
            return light_uptake, nitrogen_uptake, growth, sum_of_fluxes
        except:
            pass

In [ ]:
"""
Perform the sampling
"""
#Reproducibility
random.seed(123)

#Lists to store the results
res_list_n = []
res_list_light = []
res_list_biomass = []
res_list_sof = []

#Termination Condition
i = True

#Defining starting point to be run (manual)
test = res_c4[(500,6)]

start = time.time()

#Determining initial points (FVA)
starting = starting_conditions(test)

print(starting)

n_lb = starting[0]
n_ub = starting[1]
l_lb = starting[2]
l_ub = starting[3]

print("FVA is complete")

#Simulation loop

while i:
    res = random_simul_c4(light_lb=l_lb, light_ub = l_ub , nitrogen_lb = n_lb , #Initially from the FVA, later manually adjusted
                          nitrogen_ub = n_ub,sum_of_flux = test[0],biomass = test[1] , c4_model=c4_model)
    if res != None:
        res_list_light.append(res[0])
        res_list_n.append(res[1])
        res_list_biomass.append(res[2])
        res_list_sof.append(res[3])
    if len(res_list_n) == 500: #Number of viable solutions to be kept
        i = False

end = time.time()
print(f"This simulation lasted {(end - start)/60} minutes")
print(f"This simulation lasted {(end - start)} seconds")


In [ ]:
"""Lists with results"""
print(res_list_light)
print(res_list_n)
print(res_list_biomass)
print(res_list_sof)


## Plots

Like it was explained aboce, the bounds determined by the FVA were not sufficient to fully map out the High Performance Area corresponding to the tested starting point in the C4 model. In order to perform this task, another simulation would be run after this one with more enlarged bounds, therefore completing the mapping of the HPA until the full "triangle" could emerge

In [ ]:

df_c4 = pd.DataFrame(list(zip(res_list_light, res_list_n, res_list_biomass, res_list_sof)), columns=["Light", "N", "Biomass", "SoF"])

sns.scatterplot(x='Light', y='N', data=df_c4)
plt.show()
sns.scatterplot(x='Biomass', y='Light', data=df_c4)
plt.show()
sns.scatterplot(x='Biomass', y='N', data=df_c4)
plt.show()
sns.scatterplot(x='SoF', y='Biomass', data=df_c4)
plt.show()
sns.scatterplot(x='SoF', y='N', data=df_c4)
plt.show()
sns.scatterplot(x='SoF', y='Light', data=df_c4)
plt.show()


In [ ]:
"""Writing results to .csv file for later plotting"""

#df_c4.to_csv(f"Sample_{list(res_c4.keys())[0]}_c4.csv", index=True)